# Battle of Neighbourhoods - Week 1 - Ruslan Ikhsanov

## Introduction/Business Problem

For my project I have chosen city of London, UK. I loved the city, and since it is multinational and multicultural, 
and idea of opening of restaurant/cafe business is tricky.

Me and friend of mine are planning to open East cusine and we have to decide the location of our future new East cousin restaurant.

Business problem statement is  - 
#### Where should me and my partner open the East cousin restaurant in London, UK?

## Data

Data for the project will be taken from Wikipedia page and normalized usign Foursquare

#### Acquiring Wikipedia Dataset 

In [22]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [23]:
website_url = 'https://en.wikipedia.org/wiki/List_of_areas_of_London'
tmp = requests.get(website_url).text
soup = BeautifulSoup(tmp,'lxml')
London_table = soup.find('table',{'class':'wikitable sortable'})

In [24]:
rows = London_table.find_all('tr')

In [25]:
columns = [i.text.replace('\n', '')
           for i in rows[0].find_all('th')]

In [26]:
df = pd.DataFrame(columns = columns)

In [27]:
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df = df.append(pd.Series(values, index = columns), ignore_index = True)

        df

In [28]:
df.head(10)

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [1]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[2]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[2],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[2],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
5,Aldborough Hatch,Redbridge[3],ILFORD,IG2,020,TQ455895
6,Aldgate,City[4],LONDON,EC3,020,TQ334813
7,Aldwych,Westminster[4],LONDON,WC2,020,TQ307810
8,Alperton,Brent[5],WEMBLEY,HA0,020,TQ185835
9,Anerley,Bromley[5],LONDON,SE20,020,TQ345695


In [29]:
df = df.rename(index=str, columns = {'Location': 'Location', 'London\xa0borough': 'Borough', 'Post town': 'Post-town', 'Postcode\xa0district': 'Postcode', 'Dial\xa0code': 'Dial-code', 'OS grid ref': 'OSGridRef'})

In [30]:
df['Borough'] = df['Borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))

In [31]:
df0 = df.drop('Postcode', axis=1).join(df['Postcode'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('Postcode'))

In [32]:
df1 = df0[['Location', 'Borough', 'Postcode', 'Post-town']].reset_index(drop=True)

In [33]:
df1.head()

,Location,Borough,Postcode,Post-town
0,Abbey Wood,"Bexley, Greenwich",SE2,LONDON
1,Acton,"Ealing, Hammersmith and Fulham",W3,LONDON
2,Acton,"Ealing, Hammersmith and Fulham",W4,LONDON
3,Angel,Islington,EC1,LONDON
4,Angel,Islington,N1,LONDON


In [34]:
df2 = df1

In [35]:
df2 = df2[df2['Post-town'].str.contains('LONDON')]

In [37]:
df2.head()

,Location,Borough,Postcode,Post-town
0,Abbey Wood,"Bexley, Greenwich",SE2,LONDON
1,Acton,"Ealing, Hammersmith and Fulham",W3,LONDON
2,Acton,"Ealing, Hammersmith and Fulham",W4,LONDON
3,Angel,Islington,EC1,LONDON
4,Angel,Islington,N1,LONDON


In [38]:
df3 = df2[['Location', 'Borough', 'Postcode']].reset_index(drop=True)

In [39]:
df_london = df3

In [42]:
!pip -q install geocoder
import geocoder

In [43]:
'''Geocoder starts here'''
'''Defining a function to use --> get_latlng()'''
def get_latlng(arcgis_geocoder):
    
    # Initialize the Location (lat. and long.) to "None"
    lat_lng_coords = None
    
    # While loop helps to create a continous run until all the location coordinates are geocoded
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, London, United Kingdom'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords
'''Geocoder ends here'''


'Geocoder ends here'

In [44]:
postal_codes = df_london['Postcode']    
coordinates = [get_latlng(postal_code) for postal_code in postal_codes.tolist()]

In [45]:
df_london_loc = df_london

df_london_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_london_loc['Latitude'] = df_london_coordinates['Latitude']
df_london_loc['Longitude'] = df_london_coordinates['Longitude']

In [46]:
df_london_loc.head()

,Location,Borough,Postcode,Latitude,Longitude
0,Abbey Wood,"Bexley, Greenwich",SE2,51.49245,0.12127
1,Acton,"Ealing, Hammersmith and Fulham",W3,51.51324,-0.26746
2,Acton,"Ealing, Hammersmith and Fulham",W4,51.48944,-0.26194
3,Angel,Islington,EC1,51.52361,-0.09877
4,Angel,Islington,N1,51.53791,-0.09984
